In [15]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import random
import json
from datetime import datetime, timedelta

In [16]:
# Load the environment variables from the .env file
# In this .env, it contains openai's API Key.
load_dotenv()
# Load the key to call the client.
client = OpenAI()

In [17]:
# 
def get_completion(sys_prompt, prompt, model="gpt-4o-mini"):
    '''
    prompt: 
    model: 
    '''
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": prompt}
        ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]
    # return completion.choices[0].message.content

In [34]:
pseudo_labels = "./data/pseudo_email_labels_refined_manually.json"
with open(pseudo_labels, 'r') as file:
    data = json.load(file)

# Now 'data' holds the content of the JSON file as a Python dictionary
print(data[0])

{'Spam': 'No', 'Subject': 'Briefint session changed to 9:30 this morning', 'Sender': "Mia O'Connor", 'Time_Sensitive': 'Yes', 'Start': '2024-11-11 09:30', 'End': '2024-11-11 10:00', 'Type': 'Event', 'Category': 'Work', 'Format': 'Online', 'Location': 'Microsoft Teams', 'Action_Required': 'Yes', 'Priority_Level': 'Urgent'}


In [42]:
def label_to_prompt(entry):
    email_info = f"""
    You will write this email about {entry['Category']} as {entry['Sender']}. \
    The subject of the email is: "{entry['Subject']}"; \
    Is this email time-sensitive? {entry['Time_Sensitive']}; \
    (Don't mention time-sensitivity explicitly); \
    The session will start at {entry['Start']} and end at {entry['End']}. \
    (If the timestamp is given, make sure to include the date and time in the email in a natural way.) \
    This is a {entry['Format']} item held in {entry['Location']}. \
    Do you expect any kind of action (like reply, submission or others)? {entry['Action_Required']}. \
    (Ensure the email requests the recipients to take action if required (e.g., reply, confirm attendance, or ask questions).) \
    The urgency of this email is {entry['Priority_Level']}.
    """
    return email_info

## 1. Manually modify the emails
## 2. Use labeled real emails as prompt as well
## 3. Finish the prompt, test with GPT4o

In [ ]:
eg_email_prompt0 = label_to_prompt(data[0])
eg_email0 = f"""
Subject: Briefing Session Changed to 9:30 This Morning

Dear Team,

Please be informed that the briefing session scheduled for today has been rescheduled. The session will now take place online via Microsoft Teams from 9:30 AM to 10:00 AM on Monday, November 11, 2024.
To ensure everything runs smoothly, kindly confirm your attendance or reach out if you have any questions or concerns about accessing the session.
Thank you for your prompt attention to this update.

Best regards,
Mia O’Connor
"""

In [ ]:
eg_email_prompt1 = label_to_prompt(data[10])
eg_email1 = f"""
Subject: Coffee Catch-Up with Hugo on November 16th!

Dear All,

I hope this message finds you well! I’d love to invite you to a coffee catch-up on Saturday, November 16, 2024, from 9:00 AM to 10:45 AM. We’ll meet in person at Starbucks in the Student Union
This is a relaxed and informal gathering—feel free to drop by and enjoy a chat over coffee. No RSVP is required; just bring your great vibes and let’s have a wonderful time!
Looking forward to seeing you there.

Warm regards,
Hugo Dupont
"""

In [51]:
eg_email_prompt1

'\n    You will write this email about Leisure as Hugo Dupont.     The subject of the email is: "Coffee Catch-Up with Hugo on November 16th!";     Is this email time-sensitive? Yes;     (Don\'t mention time-sensitivity explicitly);     The session will start at 2024-11-16 09:00 and end at 2024-11-16 10:45.     (If the timestamp is given, make sure to include the date and time in the email in a natural way.)     This is a In-person item held in Starbucks in Student Union.     Do you expect any kind of action (like reply, submission or others)? No.     (Ensure the email requests the recipients to take action if required (e.g., reply, confirm attendance, or ask questions).)     The urgency of this email is Low.\n    '

In [ ]:
system_prompt = f"""
You are a professional email writer with expertise in crafting clear, concise,\
and engaging emails for different purposes. You can write formal business emails, \
friendly personal emails, academic based emails, and much more. \
Your tone, structure, and content adapt based on the context and audience. \
Be as nature as possible.
"""

task_prompt = f"""
Your task is to write an email based on a brief conclusion message. Here is an example: \
Email Conclusion: {eg_email_prompt0} \
Email: {eg_email0}; \
Email Conclusion: {eg_email_prompt1} \
Email: {eg_email1}
Email Conclusion: {eg_email_prompt_current} \
Email: 
"""